In [ ]:
#!pip install -U -q PyDrive


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("vispapers.csv")
df.head()

,Conference,Year,Title,DOI,Link,FirstPage,LastPage,PaperType,Abstract,AuthorNames-Deduped,...,Award,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,Vis,1990.0,Interdisciplinary visualization: lessons learn...,10.0000/00000002,http://dl.acm.org/citation.cfm?id=949606&CFID=...,457,457,M,NaN,Donna J. Cox,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Vis,1990.0,Surface representations of two- and three-dime...,10.1109/VISUAL.1990.146359,http://dx.doi.org/10.1109/VISUAL.1990.146359,6,"13, 460",C,The use of critical point analysis to generate...,James Helman;Lambertus Hesselink,...,BP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Vis,1990.0,FAST: a multi-processed environment for visual...,10.1109/VISUAL.1990.146360,http://dx.doi.org/10.1109/VISUAL.1990.146360,14,"27, 461-2",C,The authors discuss FAST (flow analysis softwa...,Gordon V. Bancroft;Fergus Merritt;Todd Plessel...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Vis,1990.0,The VIS-5D system for easy interactive visuali...,10.1109/VISUAL.1990.146361,http://dx.doi.org/10.1109/VISUAL.1990.146361,28,"35, 462",C,The VIS-5D system provides highly interactive ...,William L. Hibbard;David A. Santek,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Vis,1990.0,A procedural interface for volume rendering,10.1109/VISUAL.1990.146362,http://dx.doi.org/10.1109/VISUAL.1990.146362,36,"44, 462",C,"The author presents a simple, procedural inter...",James L. Montine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")

NameError: name 'doc' is not defined

In [4]:
#df['Year'] = df['Year'].astype(int)
#df = df[(df.Year > 2008) & (df.Conference.isin(['InfoVis','VAST']))]
df = df[df.Conference.isin(['InfoVis','VAST','SciVis','Vis'])]
df.shape

(3101, 27)

In [5]:
import numpy as np

abstracts = [_ for _ in df['Title'] if not _ == "NaN"]

# Simple: TF-IDF + UMAP

In [6]:
## 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the data and create document vectors
corpus = abstracts
tfidf = TfidfVectorizer(min_df=5)

X = tfidf.fit_transform(corpus)
#y = df['Bias']

# Create the visualizer and draw the vectors
# tsne = TSNEVisualizer()
# tsne.fit(X, y)

import umap
reducer = umap.UMAP()
X_embedded = reducer.fit_transform(X)

# Instantiate the text visualizer
# umap = UMAPVisualizer()
# umap.fit(X, y)
# umap.show()

In [7]:
title = [_ for _ in df['Title']]
authors = [_ for _ in df['AuthorNames-Deduped']]
conference = [_ for _ in df['Conference']]
year = [_ for _ in df['Year']]

embed_docs = pd.DataFrame(X_embedded, columns=['X','Y'])
embed_docs['Title'] = title
embed_docs['Authors'] = authors
embed_docs['Conference'] = conference
embed_docs['Year'] = year
#embed_docs['Bias'] = embed_docs['Authors'].str.contains(u'Ryan Wesslen|Emily Wall')
# embed_words = embed_words.reset_index()
embed_docs.head(n=10)

,X,Y,Title,Authors,Conference,Year
0,-3.451567,-0.519439,Interdisciplinary visualization: lessons learn...,Donna J. Cox,Vis,1990.0
1,-0.999156,4.293102,Surface representations of two- and three-dime...,James Helman;Lambertus Hesselink,Vis,1990.0
2,4.153953,-2.362112,FAST: a multi-processed environment for visual...,Gordon V. Bancroft;Fergus Merritt;Todd Plessel...,Vis,1990.0
3,0.720643,-0.656313,The VIS-5D system for easy interactive visuali...,William L. Hibbard;David A. Santek,Vis,1990.0
4,1.028778,-2.213217,A procedural interface for volume rendering,James L. Montine,Vis,1990.0
5,0.440015,-1.910640,Techniques for the interactive visualization o...,Gregory M. Nielson;Bernd Hamann,Vis,1990.0
6,-1.264380,4.324491,Displaying voxel-based objects according to th...,Yaser Yacoob,Vis,1990.0
7,-1.049132,4.289662,Interpreting a 3D object from a rough 2D line ...,Del Lamb;Amit Bandopadhay,Vis,1990.0
8,-1.063875,4.290065,Animation techniques for chain-coded objects,Anthony J. Maeder,Vis,1990.0
9,-1.270250,4.429224,Extracting geometric models through constraint...,James V. Miller;David E. Breen;Michael J. Wozny,Vis,1990.0


In [8]:
import altair as alt

# Brush for selection
brush = alt.selection(type='interval')

points = alt.Chart(embed_docs).mark_circle(size=40).encode(
    x='X',
    y='Y',
    color = alt.condition(brush, 'Conference', alt.value('grey')),
    tooltip = ['Title', 'Authors', 'Conference', 'Year']
).add_selection(brush).properties(
    width=600,
    height=500
)

# Base chart for data tables
ranked_text = alt.Chart(embed_docs).mark_text().encode(
    y=alt.Y('row_number:O',axis=None)
).transform_window(
    row_number='row_number()'
).transform_filter(
    brush
).transform_window(
    rank='rank(row_number)'
).transform_filter(
    alt.datum.rank<20
)

# Data Tables
title = ranked_text.encode(text='Title:N').properties(title='Title')
authors = ranked_text.encode(text='Authors:N').properties(title='Authors')
conference = ranked_text.encode(text='Conference:N').properties(title='Conference')
text = alt.hconcat(title, authors, conference) # Combine data tables

# Build chart
chart = alt.vconcat(
    points,
    text
).resolve_legend(
    color="independent"
)

chart

alt.VConcatChart(...)

# Flair Embeddings

In [ ]:
# https://www.analyticsvidhya.com/blog/2019/02/flair-nlp-library-python/

# !pip install -q matplotlib-venn
# !pip install torch
# import torch
# !pip install flair
# import flair

In [15]:
# https://www.analyticsvidhya.com/blog/2017/06/word-embeddings-count-word2veec/
from flair.embeddings import WordEmbeddings
#from flair.embeddings import CharacterEmbeddings
from flair.embeddings import StackedEmbeddings
from flair.embeddings import FlairEmbeddings
#from flair.embeddings import BertEmbeddings
#from flair.embeddings import ELMoEmbeddings

### Initialising embeddings (un-comment to use others) ###
glove_embedding = WordEmbeddings('glove')
#character_embeddings = CharacterEmbeddings()
flair_forward  = FlairEmbeddings('news-forward-fast')
flair_backward = FlairEmbeddings('news-backward-fast')
#bert_embedding = BertEmbedding('bert-base-uncased')
#elmo_embedding = ElmoEmbedding()

stacked_embeddings = StackedEmbeddings(embeddings = [glove_embedding, flair_forward, flair_backward])

2019-12-20 18:55:48,563 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmpt3hbhobc


100%|██████████| 160000128/160000128 [00:08<00:00, 19602984.44B/s]

2019-12-20 18:55:57,238 copying /tmp/tmpt3hbhobc to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2019-12-20 18:55:57,630 removing temp file /tmp/tmpt3hbhobc
2019-12-20 18:55:58,111 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim not found in cache, downloading to /tmp/tmpfc9hf3o_


100%|██████████| 21494764/21494764 [00:01<00:00, 12418343.88B/s]

...

100%|██████████| 19689779/19689779 [00:01<00:00, 12136632.71B/s]

2019-12-20 18:56:04,633 copying /tmp/tmp1mc55j2q to cache at /root/.flair/embeddings/lm-news-english-forward-1024-v0.2rc.pt
2019-12-20 18:56:04,660 removing temp file /tmp/tmp1mc55j2q


2019-12-20 18:56:05,567 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-backward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmpgov2i2j3


100%|██████████| 19689779/19689779 [00:01<00:00, 12808261.88B/s]

2019-12-20 18:56:07,625 copying /tmp/tmpgov2i2j3 to cache at /root/.flair/embeddings/lm-news-english-backward-1024-v0.2rc.pt
2019-12-20 18:56:07,657 removing temp file /tmp/tmpgov2i2j3


In [ ]:
%%time
# 0:11 = 18 seconds
# 0:21 = 37.5 seconds 
# 0:41 = 68 seconds
# ~900 estimate to be about 30 minutes

from flair.embeddings import DocumentPoolEmbeddings
from flair.data import Sentence
# create a sentence #

document_embeddings = DocumentPoolEmbeddings([glove_embedding,
                                              flair_backward,
                                              flair_forward],
                                              pooling='mean')

docs=[]
for i in abstracts:
  sentence = Sentence(i)

  # create document embeddings
  document_embeddings.embed(sentence)

  v = sentence.get_embedding()
  v = v.detach().numpy()
  docs.append(v)

doc_embed = pd.DataFrame(docs)

doc_embed.to_csv("drive/My Drive/doc-embeddings.csv")

CPU times: user 29min 45s, sys: 4.24 s, total: 29min 49s
Wall time: 29min 51s


In [16]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [ ]:
doc_embed = pd.read_csv("drive/My Drive/doc-embeddings.csv")
doc_embed = doc_embed.drop(['Unnamed: 0'], axis =1)


In [ ]:
import umap
import numpy as np
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
text_umap = Pipeline([
    ('svd', TruncatedSVD(n_components=50)),
    ('umap', umap.UMAP()),
])

X_embedded = text_umap.fit_transform(doc_embed.to_numpy())

In [ ]:
# import umap
# reducer = umap.UMAP()
# X_embedded = reducer.fit_transform(doc_embed.to_numpy())

In [29]:
import altair as alt
title = [_ for _ in df['Title']]
authors = [_ for _ in df['AuthorNames-Deduped']]
conference = [_ for _ in df['Conference']]
year = [_ for _ in df['Year']]

embed_docs = pd.DataFrame(X_embedded, columns=['X','Y'])
embed_docs['Title'] = title
embed_docs['Authors'] = authors
embed_docs['Conference'] = conference
embed_docs['Year'] = year
embed_docs['Bias'] = embed_docs['Authors'].str.contains(u'Ryan Wesslen|Emily Wall')
# embed_words = embed_words.reset_index()
embed_docs.head(n=10)


,X,Y,Title,Authors,Conference,Year,Bias
0,3.604155,1.125983,"""Search, Show Context, Expand on Demand"": Supp...",Frank van Ham;Adam Perer,InfoVis,2009,False
1,0.996285,0.436554,A Comparison of User-Generated and Automatic G...,Tim Dwyer;Bongshin Lee;Danyel Fisher;Kori Inkp...,InfoVis,2009,False
2,1.056073,2.072692,A Multi-Threading Architecture to Support Inte...,Harald Piringer;Christian Tominski;Philipp Mui...,InfoVis,2009,False
3,2.973564,1.865767,A Nested Model for Visualization Design and Va...,Tamara Munzner,InfoVis,2009,False
4,0.313426,3.420837,ABySS-Explorer: Visualizing Genome Sequence As...,Cydney B. Nielsen;Shaun D. Jackman;Inanç Birol...,InfoVis,2009,False
5,1.834247,1.864681,ActiviTree: Interactive Visual Exploration of ...,Katerina Vrotsou;Jimmy Johansson;Matthew D. Co...,InfoVis,2009,False
6,1.322334,0.108733,Bubble Sets: Revealing Set Relations with Isoc...,Christopher Collins;Gerald Penn;Sheelagh Carpe...,InfoVis,2009,False
7,3.090238,0.735532,code_swarm: A Design Study in Organic Software...,Michael Ogawa;Kwan-Liu Ma,InfoVis,2009,False
8,-1.290576,0.830037,Comparing Dot and Landscape Spatializations fo...,Melanie Tory;Colin Swindells;Rebecca Dreezer,InfoVis,2009,False
9,0.804917,1.441535,Configuring Hierarchical Layouts to Address Re...,Aidan Slingsby;Jason Dykes;Jo Wood,InfoVis,2009,False


In [30]:
import altair as alt

# Brush for selection
brush = alt.selection(type='interval')

points = alt.Chart(embed_docs).mark_circle(size=40).encode(
    x='X',
    y='Y',
    color = alt.condition(brush, 'Bias', alt.value('grey')),
    tooltip = ['Title', 'Authors', 'Conference', 'Year']
).add_selection(brush).properties(
    width=600,
    height=500
)

# Base chart for data tables
ranked_text = alt.Chart(embed_docs).mark_text().encode(
    y=alt.Y('row_number:O',axis=None)
).transform_window(
    row_number='row_number()'
).transform_filter(
    brush
).transform_window(
    rank='rank(row_number)'
).transform_filter(
    alt.datum.rank<20
)

# Data Tables
title = ranked_text.encode(text='Title:N').properties(title='Title')
authors = ranked_text.encode(text='Authors:N').properties(title='Authors')
conference = ranked_text.encode(text='Conference:N').properties(title='Conference')
text = alt.hconcat(title, authors, conference) # Combine data tables

# Build chart
chart = alt.vconcat(
    points,
    text
).resolve_legend(
    color="independent"
)

chart

alt.VConcatChart(...)

In [9]:
chart.save('chart.html')

# Word-Level Embeddings

In [ ]:
#https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_5_DOCUMENT_EMBEDDINGS.md
from flair.embeddings import DocumentPoolEmbeddings, Sentence
from flair.data import Sentence
# create a sentence #
sentence = Sentence(abstracts[0])

# just embed a sentence using the StackedEmbedding as you would with any single embedding.
stacked_embeddings.embed(sentence)

In [ ]:
v=[]
i=[]
# now check out the embedded tokens.
for token in sentence:
    i.append(token)
    v.append(token.embedding.numpy())

px = pd.DataFrame(v, index=i)


# Document Embeddings

In [ ]:
for token in sentence:
    print(token)

Token: 1 The
Token: 2 authors
Token: 3 discuss
Token: 4 FAST
Token: 5 (flow
Token: 6 analysis
Token: 7 software
Token: 8 toolkit),
Token: 9 an
Token: 10 implementation
Token: 11 of
Token: 12 a
Token: 13 software
Token: 14 system
Token: 15 for
Token: 16 fluid
Token: 17 mechanics
Token: 18 analysis.
Token: 19 Visualization
Token: 20 of
Token: 21 computational
Token: 22 aerodynamics
Token: 23 requires
Token: 24 flexible,
Token: 25 extensible,
Token: 26 and
Token: 27 adaptable
Token: 28 software
Token: 29 tools
Token: 30 for
Token: 31 performing
Token: 32 analysis
Token: 33 tasks.
Token: 34 An
Token: 35 overview
Token: 36 of
Token: 37 FAST
Token: 38 is
Token: 39 given,
Token: 40 and
Token: 41 its
Token: 42 architecture
Token: 43 is
Token: 44 discussed.
Token: 45 Interactive
Token: 46 visualization
Token: 47 control
Token: 48 is
Token: 49 addressed.
Token: 50 The
Token: 51 advantages
Token: 52 and
Token: 53 disadvantages
Token: 54 of
Token: 55 FAST
Token: 56 are
Token: 57 discussed.&lt;&lt;

In [ ]:
document_embeddings = DocumentPoolEmbeddings([glove_embedding,
                                             flair_backward,
                                             flair_backward],
                                             pooling='min')

In [ ]:
from flair.models import SequenceTagger
# load the NER tagger
tagger = SequenceTagger.load('ner')

# run NER over sentence
tagger.predict(sentence)
#Done! The Sentence now has entity annotations. Print the sentence to see what the tagger found.

print(sentence)
print('The following NER tags are found:')

# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)

2019-12-15 18:01:00,456 loading file /root/.flair/models/en-ner-conll03-v0.4.pt
Sentence: "The authors discuss FAST (flow analysis software toolkit), an implementation of a software system for fluid mechanics analysis. Visualization of computational aerodynamics requires flexible, extensible, and adaptable software tools for performing analysis tasks. An overview of FAST is given, and its architecture is discussed. Interactive visualization control is addressed. The advantages and disadvantages of FAST are discussed.&lt;&lt;ETX&gt;&gt;" - 57 Tokens
The following NER tags are found:


In [ ]:
sentence = Sentence(abstracts[200])

# predict NER tags
tagger.predict(sentence)

# print sentence with predicted tags
for entity in sentence.get_spans('ner'):
    print(entity)

MISC-span [63]: "Eulerian"
MISC-span [68]: "Lagrangian"
MISC-span [114]: "Lagrangian"


In [ ]:
from flair.embeddings import OneHotEmbeddings

embeddings = OneHotEmbeddings(corpus)

# document pool embeddings
document_embeddings = DocumentPoolEmbeddings([embeddings], fine_tune_mode='none')

NameError: ignored

In [ ]:
z = sentence.embedding.size()[1]

### Vectorising text ###
# creating a tensor for storing sentence embeddings
s = torch.zeros(0,z)
# iterating Sentences #
for tweet in tqdm(txt):   
  sentence = Sentence(tweet)
  document_embeddings.embed(sentence)
  # Adding Document embeddings to list #
  s = torch.cat((s, sentence.embedding.view(-1,z)),0)